In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
sys.path.append('/Users/Tim/PycharmProjects/HOI/')

In [20]:
import sys
sys.path

['/Users/tim/PycharmProjects/HOI/examples',
 '/Users/tim/opt/anaconda3/envs/hoi/lib/python310.zip',
 '/Users/tim/opt/anaconda3/envs/hoi/lib/python3.10',
 '/Users/tim/opt/anaconda3/envs/hoi/lib/python3.10/lib-dynload',
 '',
 '/Users/tim/opt/anaconda3/envs/hoi/lib/python3.10/site-packages',
 '/Users/Tim/PycharmProjects/HOI/',
 '/Users/Tim/PycharmProjects/HOI/']

In [33]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [34]:
from synthetic_data import make_iid_example
from HOI.preprocessings import compute_kernel
from HOI.tests import test_independence

In [7]:
# power calculation for higher-order data
power ={}
for d in tqdm(np.arange(0.1, 1, 0.1)):
    rejects = 0
    for i in np.arange(100):
        df = make_iid_example('higher-order', s = d, n_sample=100)
        data_dict, kernel_dict = compute_kernel(df)
        _, _, reject = test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']],
                                         [data_dict['d1'], data_dict['d2'], data_dict['d3']],
                                          mode = 'iid', n_perms=5000, alpha=0.05)
        rejects = rejects + reject
    power[str(d)] = rejects/100

100%|█████████████████████████████████████████████| 9/9 [12:27<00:00, 83.04s/it]


In [8]:
power

{'0.1': 0.09,
 '0.2': 0.09,
 '0.30000000000000004': 0.16,
 '0.4': 0.19,
 '0.5': 0.58,
 '0.6': 0.97,
 '0.7000000000000001': 1.0,
 '0.8': 1.0,
 '0.9': 1.0}

In [ ]:
# power calculation for multi-normal data
power ={}
for d in tqdm(np.arange(0.1, 1, 0.1)):
    rejects = 0
    for i in np.arange(100):
        df = make_iid_example(mode='multi-normal', s = d, n_sample=100)
        data_dict, kernel_dict = compute_kernel(df)
        _, _, reject1 = test_independence([kernel_dict['d1'], kernel_dict['d2']],
                                         [data_dict['d1'], data_dict['d2']],
                                          mode = 'iid', n_perms=5000, alpha=0.05)
        
        _, _, reject2 = test_independence([kernel_dict['d1'], kernel_dict['d3']],
                                 [data_dict['d1'], data_dict['d3']],
                                  mode = 'iid', n_perms=5000, alpha=0.05)

        _, _, reject3 = test_independence([kernel_dict['d2'], kernel_dict['d3']],
                             [data_dict['d2'], data_dict['d3']],
                              mode = 'iid', n_perms=5000, alpha=0.05)
        
        rejects = rejects + reject1+reject2+reject3
    power[str(d)] = rejects/300

 56%|████████████████████████▍                   | 5/9 [12:52<10:15, 153.86s/it]

In [ ]:
power

In [ ]:
# power calculation for interpolated data
power ={}
for d in tqdm(np.arange(0.1, 1, 0.1)):
    rejects = 0
    for i in np.arange(100):
        df = make_iid_example(mode='interpolated', s = d, n_sample=100)
        data_dict, kernel_dict = compute_kernel(df)
        _, _, reject1 = test_independence([kernel_dict['d1'], kernel_dict['d2']],
                                         [data_dict['d1'], data_dict['d2']],
                                          mode = 'iid', n_perms=5000, alpha=0.05)
        
        _, _, reject2 = test_independence([kernel_dict['d1'], kernel_dict['d3']],
                                 [data_dict['d1'], data_dict['d3']],
                                  mode = 'iid', n_perms=5000, alpha=0.05)

        _, _, reject3 = test_independence([kernel_dict['d2'], kernel_dict['d3']],
                             [data_dict['d2'], data_dict['d3']],
                              mode = 'iid', n_perms=5000, alpha=0.05)
        
        rejects = rejects + reject1+reject2+reject3
    power[str(d)] = rejects/300

In [ ]:
power

# multi-normal, S = 0.99

In [147]:
data_df = make_iid_example('multi-normal', s = 0.99)

In [148]:
data_df

,d1,d2,d3
0,0.975709,1.003210,0.737326
1,1.239862,1.194489,1.066782
2,-0.700289,-0.519349,-0.414743
3,0.453285,0.337193,0.322808
4,-1.582181,-1.615040,-1.683302
...,...,...,...
95,-0.428175,-0.371847,-0.496028
96,0.268221,0.377493,0.239660
97,0.832519,0.840402,0.768648
98,-0.837189,-0.848547,-0.419025


In [149]:
data_dict, kernel_dict = compute_kernel(data_df)

In [150]:
kernel_dict['d1']

array([[1.        , 0.9637786 , 0.22645637, ..., 0.98921774, 0.17591686,
        0.29733694],
       [0.9637786 , 1.        , 0.13665967, ..., 0.916006  , 0.10217753,
        0.18770997],
       [0.22645637, 0.13665967, 1.        , ..., 0.28872924, 0.99013952,
        0.98631723],
       ...,
       [0.98921774, 0.916006  , 0.28872924, ..., 1.        , 0.2289899 ,
        0.36994737],
       [0.17591686, 0.10217753, 0.99013952, ..., 0.2289899 , 1.        ,
        0.95403454],
       [0.29733694, 0.18770997, 0.98631723, ..., 0.36994737, 0.95403454,
        1.        ]])

In [155]:
test_independence([kernel_dict['d1'], kernel_dict['d2']], n_perms=1000, alpha=0.05)

(0.08310243605319323, 0.00429274471233454, 1)

In [156]:
test_independence([kernel_dict['d1'], kernel_dict['d3']], n_perms=1000, alpha=0.05)

(0.08174070105053433, 0.004197527769259968, 1)

In [157]:
test_independence([kernel_dict['d2'], kernel_dict['d3']], n_perms=1000, alpha=0.05)

(0.08084666712698341, 0.004234556705420722, 1)

In [158]:
test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']], n_perms=1000, alpha=0.05)

(0.13314619215894336, 0.0063933156678924186, 1)

# multi-normal data, S = 0.01

In [13]:
data_df = make_iid_example('multi-normal', s = 0.01)

In [16]:
data_dict, kernel_dict = compute_kernel(data_df)

In [17]:
test_independence([kernel_dict['d1'], kernel_dict['d2']], n_perms=100, alpha=0.05)

(0.0017278482088732838, 0.004842012134713469, 0)

In [18]:
test_independence([kernel_dict['d1'], kernel_dict['d3']], n_perms=100, alpha=0.05)

(0.0016071037474718608, 0.004114489512150432, 0)

In [19]:
test_independence([kernel_dict['d3'], kernel_dict['d2']], n_perms=100, alpha=0.05)

(0.001198240010647833, 0.003850132562742803, 0)

In [20]:
test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']], n_perms=50, alpha=0.05)

(0.0032659564347825576, 0.007843157200729367, 0)

# higher-order data; S = 0.5

In [83]:
data_df = make_iid_example('higher-order', s = 0.6)

In [84]:
data_dict, kernel_dict = compute_kernel(data_df)

In [85]:
test_independence([kernel_dict['d1'], kernel_dict['d2']], n_perms=100, alpha=0.05)

(0.0006944484189721489, 0.004337485209658709, 0)

In [86]:
test_independence([kernel_dict['d1'], kernel_dict['d3']], n_perms=100, alpha=0.05)

(0.0012747768307277418, 0.004688985410978508, 0)

In [87]:
test_independence([kernel_dict['d3'], kernel_dict['d2']], n_perms=100, alpha=0.05)

(0.001522406258980591, 0.004121995671024448, 0)

In [88]:
test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']], n_perms=50, alpha=0.05)

(0.007368907157075333, 0.0063171277642197765, 1)

# interpolated; S = 0.3

In [95]:
data_df = make_iid_example('interpolated', s = 0.3)

In [96]:
data_dict, kernel_dict = compute_kernel(data_df)

In [97]:
test_independence([kernel_dict['d1'], kernel_dict['d2']], n_perms=100, alpha=0.05)

(0.0024423595474469817, 0.004647639711442664, 0)

In [98]:
test_independence([kernel_dict['d1'], kernel_dict['d3']], n_perms=100, alpha=0.05)

(0.005089925274446072, 0.0034471508094522374, 1)

In [99]:
test_independence([kernel_dict['d3'], kernel_dict['d2']], n_perms=100, alpha=0.05)

(0.0025824951419427533, 0.00411072069783569, 0)

In [100]:
test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']], n_perms=50, alpha=0.05)

(0.005947573691364005, 0.006876821572011083, 0)

# interpolated; S = 0.42

In [159]:
data_df = make_iid_example('interpolated', s = 0.42)

In [160]:
data_dict, kernel_dict = compute_kernel(data_df)

In [161]:
test_independence([kernel_dict['d1'], kernel_dict['d2']], n_perms=1000, alpha=0.05)

(0.002752515156401958, 0.00419199666853276, 0)

In [162]:
test_independence([kernel_dict['d1'], kernel_dict['d3']], n_perms=1000, alpha=0.05)

(0.001399464528044625, 0.0038643192170724205, 0)

In [163]:
test_independence([kernel_dict['d3'], kernel_dict['d2']], n_perms=1000, alpha=0.05)

(0.0054425068191708, 0.004138935279364153, 1)

In [164]:
test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']], n_perms=1000, alpha=0.05)

(0.005447645936407619, 0.006324826755856772, 0)

# interpolated; S = 0.42

In [42]:
data_df = make_iid_example('interpolated', s = 0.3)

In [43]:
data_dict, kernel_dict = compute_kernel(data_df)

In [44]:
test_independence([kernel_dict['d1'], kernel_dict['d2']], n_perms=5000, alpha=0.05)

(0.003489291628939184, 0.003844612210234244, 0)

In [45]:
test_independence([kernel_dict['d1'], kernel_dict['d3']], n_perms=5000, alpha=0.05)

(0.002444563781906939, 0.003849271676652277, 0)

In [46]:
test_independence([kernel_dict['d3'], kernel_dict['d2']], n_perms=5000, alpha=0.05)

(0.002993437834578505, 0.00396435225818581, 0)

In [47]:
test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']], n_perms=5000, alpha=0.05)

(0.0060342094792866785, 0.006228593903612412, 0)